## RNN

### 01 라이브러리, 하이퍼파라미터

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from torchvision import transforms
from torchvision.datasets import FashionMNIST

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classification_data_path = "./sample_data/"

test_data_ratio = 0.3

random_state = 0

sequence_length = 28

classifier_input_size = 28
classifier_hidden_size = 128
classifier_output_size = 10

num_layer = 4

learning_rate = 1e-4 # 5e-3
epochs = 50
batch_size = 32

classification_criterion = nn.CrossEntropyLoss()

### 02 Classification, Data Loader

In [5]:
class ClassificationDataLoader:
    def __init__(self, data_path, batch_size):
      self.data_path = data_path
      self.batch_size = batch_size
      self.transform = transforms.ToTensor()

    def __call__(self, flag):
        if flag == "train":
            dataset = FashionMNIST(
                self.data_path, train=True, download=True, transform=self.transform
            )
        else:
            dataset = FashionMNIST(
                self.data_path, train=False, download=True, transform=self.transform
            )

        data_loader = DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=0,
        )

        return data_loader


classification_loader = ClassificationDataLoader(classification_data_path, batch_size)
classification_train_loader = classification_loader("train")
classification_test_loader = classification_loader("test")

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 201kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.79MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.33MB/s]


### 03 RNN 정의하기

In [6]:
class RecurrentNeuralNetwork(nn.Module): # PyTorch의 모델 구조 정의를 위해 nn.Module을 상속
    def __init__(
        # input_size         RNN에 입력되는 벡터의 길이 (ex: 28)
        # sequence_length    시퀀스 길이, 즉 time step 수 (ex: 28)
        # hidden_size        RNN의 hidden state 벡터 차원
        # num_layer          RNN 층 수
        # output_size        클래스 개수 (ex: 10 for FashionMNIST)
        self, input_size, sequence_length, hidden_size, num_layer, output_size
    ):
        super(RecurrentNeuralNetwork, self).__init__() # 부모 클래스(nn.Module)의 초기화 메서드를 호출
        self.input_size = input_size
        self.sequence_length = sequence_length
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size

        # PyTorch 내장 RNN 레이어 생성
        self.rnn = nn.RNN(
            self.input_size, self.hidden_size, self.num_layer, batch_first = True
        )
        self.output_layer = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, data):
      data = self.prepare_input(data) # 이미지를 RNN에 넣기 적합한 시퀀스 형태로 변환
      extracted_feature, _ = self.rnn(data) #  두 번째 출력은 hidden_state인데 여기선 사용하지 않음 (_로 무시)
      extracted_feature = extracted_feature[:, -1, :] # 마지막 시점의 출력만 골라서 다음 층으로 보냄
      result = self.output_layer(extracted_feature)
      return result

    def prepare_input(self, data):
      data = data.view(-1, self.sequence_length, self.input_size)
      return data

In [7]:
classifier = RecurrentNeuralNetwork(
    classifier_input_size,
    sequence_length,
    classifier_hidden_size,
    num_layer,
    classifier_output_size,
).to(device)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

In [8]:
classifier

RecurrentNeuralNetwork(
  (rnn): RNN(28, 128, num_layers=4, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)

### 04 모델 학습

In [9]:
class Trainer:
    def __init__(self, model, data_loader, optimizer, criterion, epochs, device):
        self.model = model
        self.data_loader = data_loader
        self.optimizer = optimizer
        self.criterion = criterion
        self.epochs = epochs
        self.device = device

    def train(self):
      # PyTorch의 internal mode 설정: 모델을 train mode로 바꿈
      # (예: Dropout, BatchNorm 등은 train/test 모드에 따라 다르게 동작)
        self.model.train()

        for epoch in range(self.epochs):
            print(f"Epoch {epoch + 1} / {self.epochs}")
            # 학습용 DataLoader에서 (입력, 정답) 배치를 하나씩 가져옴
            for data, label in self.data_loader:
                # 이전 iteration의 gradient를 초기화
                # PyTorch는 gradient를 누적하는 방식이라 매번 비워줘야 함
                self.optimizer.zero_grad()
                # 입력 데이터와 정답 레이블을 GPU 또는 CPU로 이동
                data, label = data.to(self.device), label.to(self.device)
                result = self.model(data)
                loss = self.criterion(result, label)
                # 역전파(Backpropagation): 각 파라미터에 대한 gradient 계산
                loss.backward()
                self.optimizer.step()


classifier_trainer = Trainer(
    classifier,
    classification_train_loader,
    classifier_optimizer,
    classification_criterion,
    epochs,
    device,
)
classifier_trainer.train()


Epoch 1 / 50
Epoch 2 / 50
Epoch 3 / 50
Epoch 4 / 50
Epoch 5 / 50
Epoch 6 / 50
Epoch 7 / 50
Epoch 8 / 50
Epoch 9 / 50
Epoch 10 / 50
Epoch 11 / 50
Epoch 12 / 50
Epoch 13 / 50
Epoch 14 / 50
Epoch 15 / 50
Epoch 16 / 50
Epoch 17 / 50
Epoch 18 / 50
Epoch 19 / 50
Epoch 20 / 50
Epoch 21 / 50
Epoch 22 / 50
Epoch 23 / 50
Epoch 24 / 50
Epoch 25 / 50
Epoch 26 / 50
Epoch 27 / 50
Epoch 28 / 50
Epoch 29 / 50
Epoch 30 / 50
Epoch 31 / 50
Epoch 32 / 50
Epoch 33 / 50
Epoch 34 / 50
Epoch 35 / 50
Epoch 36 / 50
Epoch 37 / 50
Epoch 38 / 50
Epoch 39 / 50
Epoch 40 / 50
Epoch 41 / 50
Epoch 42 / 50
Epoch 43 / 50
Epoch 44 / 50
Epoch 45 / 50
Epoch 46 / 50
Epoch 47 / 50
Epoch 48 / 50
Epoch 49 / 50
Epoch 50 / 50


### 05 모델 테스트

In [10]:
class Tester:
    def __init__(self, model, data_loader, device):
        self.model = model
        self.data_loader = data_loader
        self.device = device

    def classifier_test(self):
        self.model.eval()

        pred_list, label_list = [], []

        # gradient를 계산하지 않도록 설정
        # 테스트는 학습이 아니므로 gradient 필요 없음(메모리 절약, 속도 향상)
        with torch.no_grad():
            for data, label in self.data_loader:
                data, label = data.to(self.device), label.to(self.device)
                result = self.model(data)
                # torch.max(result, 1): 클래스 확률 중 가장 높은 것 선택
                _, pred = torch.max(result, 1)
                pred_list.extend(pred.cpu().numpy())
                label_list.extend(label.cpu().numpy())

            print("Accuracy:", accuracy_score(label_list, pred_list))



In [11]:
classifier_tester = Tester(classifier, classification_test_loader, device)
classifier_tester.classifier_test()

Accuracy: 0.8798076923076923
